<a href="https://colab.research.google.com/github/dagousket/ML-course-VIB-2020/blob/master/Histone_marks_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gene activity prediction from histone mark data

#### Import env

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

random_seed = 666
np.random.seed(random_seed)

#### Import data

In [2]:
train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test.csv")

# Save labels in external variable
train_labels = train.pop("Label")
train_index_col = train.pop("GeneId")
test_index_col = test.pop("GeneId")

In [3]:
# Look at data
plt.figure(figsize=(30,10))
sns.heatmap(train.sample(20))

#### Generate new features (the fun part)

Feature 1 : max, sum and std of unormalised data for each histone marks

In [4]:
X = train.copy()

marks = list(set([c_name.split("_")[0] for c_name in X.columns]))

def get_global(X, marks):
  tmp_out = {}
  for mark in marks:
    tmp_out[mark + "_sum"] = X.loc[:,[c_name for c_name in X.columns if c_name.startswith(mark)]].sum(axis = 1)
    tmp_out[mark + "_max"] = X.loc[:,[c_name for c_name in X.columns if c_name.startswith(mark)]].max(axis = 1)
    tmp_out[mark + "_std"] = X.loc[:,[c_name for c_name in X.columns if c_name.startswith(mark)]].std(axis = 1)
    tmp_out[mark + "_sumTSS"] = X.loc[:,[mark + "_" + str(bin) for bin in range(40,60)]].sum(axis = 1)
    tmp_out[mark + "_sumUpstream"] = X.loc[:,[mark + "_" + str(bin) for bin in range(0,40)]].sum(axis = 1)
    tmp_out[mark + "_sumDownstream"] = X.loc[:,[mark + "_" + str(bin) for bin in range(60,100)]].sum(axis = 1)
    tmp_out[mark + "_maxTSS"] = X.loc[:,[mark + "_" + str(bin) for bin in range(40,60)]].max(axis = 1)
    tmp_out[mark + "_stdTSS"] = X.loc[:,[mark + "_" + str(bin) for bin in range(40,60)]].std(axis = 1)
    tmp_out[mark + "_stdUpstream"] = X.loc[:,[mark + "_" + str(bin) for bin in range(0,40)]].std(axis = 1)
    tmp_out[mark + "_stdDownstream"] = X.loc[:,[mark + "_" + str(bin) for bin in range(60,100)]].std(axis = 1)
  return pd.DataFrame(tmp_out)

get_global(X.head(), marks)

H3K36me3_sum  H3K36me3_max  ...  H3K4me3_stdUpstream  H3K4me3_stdDownstream
0           720            31  ...             0.712075               0.905468
1            79             4  ...             1.217132               1.670061
2           356            19  ...             2.232482               1.496791
3            81             4  ...             1.347124               1.933344
4           243            12  ...             0.986771               0.723241

[5 rows x 50 columns]

Feature 2 : gini score

In [5]:
def gini(x):
    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

def get_gini(X, marks):
  tmp_out = {}
  res_out = {}
  for mark in marks:
    # Do for unscaled data
    tmp_out[mark] = X.loc[:,[c_name for c_name in X.columns if c_name.startswith(mark)]]
    res_out[mark + "_gini"] = tmp_out[mark].apply(lambda y : gini(y.to_numpy()), axis = 1)
  return pd.DataFrame(res_out)

get_gini(X.head(), marks)

H3K36me3_gini  H3K27me3_gini  H3K4me1_gini  H3K9me3_gini  H3K4me3_gini
0       0.513056       0.410000      0.547353      0.625834      0.558444
1       0.603418       0.612368      0.502453      0.553516      0.446686
2       0.540225       0.537010      0.553908      0.475412      0.388069
3       0.602840       0.507213      0.434667      0.509070      0.381000
4       0.574938       0.552657      0.513803      0.764777      0.621358

Feature 3 : proximal vs. distal enrichment + upstream vs. downstream enrichment

In [6]:
from scipy import stats

def get_binom(X, marks):
  tmp_out = {}
  res_out = {}
  for mark in marks:
    tmp_out[mark] = X.loc[:,[c_name for c_name in X.columns if c_name.startswith(mark)]]
    res_out[mark + "_TSSenrich"] = tmp_out[mark].apply(lambda y : stats.binom_test(sum(y[range(40,60)]), n = sum(y)), axis = 1)
    res_out[mark + "_Upstreamenrich"] = tmp_out[mark].apply(lambda y : stats.binom_test(sum(y[range(0,40)]), n = sum(y)), axis = 1)
    res_out[mark + "_Downstreamenrich"] = tmp_out[mark].apply(lambda y : stats.binom_test(sum(y[range(60,100)]), n = sum(y)), axis = 1)
  return pd.DataFrame(res_out)

get_binom(X.head(), marks)

H3K36me3_TSSenrich  ...  H3K4me3_Downstreamenrich
0        3.733584e-50  ...              2.461064e-01
1        3.761513e-05  ...              5.454727e-01
2        5.391031e-46  ...              1.846451e-13
3        5.656215e-06  ...              4.014365e-01
4        5.062892e-04  ...              7.275502e-03

[5 rows x 15 columns]

Feature 4 : Distance TSS-Max and Polyfit at Max position on library sized data

In [7]:
def find_slope_from_bins(mybins):
  maxbin = np.argmax(mybins)
  upstream_bins = mybins[range(max(0,maxbin-15),maxbin)]
  downstream_bins = mybins[range(maxbin,min(len(mybins),maxbin+15))]
  full_bins = mybins[range(max(0,maxbin-15),min(len(mybins),maxbin+15))]
  if len(upstream_bins) > 5 :
    upSlope = np.polyfit(range(0,len(upstream_bins)),upstream_bins,1)[0]
  else :
      upSlope = 0
  if len(downstream_bins) > 5 :
    downSlope = np.polyfit(range(0,len(downstream_bins)),downstream_bins,1)[0]
  else :
    downSlope = 0
  if len(full_bins) > 10 :
    fullSlope = np.polyfit(range(0,len(full_bins)),full_bins,2)
    fullSlope2 = fullSlope[0]
    fullSlope1 = fullSlope[1]
  else :
    fullSlope2 = 0
    fullSlope1 = 0
  return [maxbin, upSlope, downSlope, fullSlope2, fullSlope1]

def get_polyfit(X, marks):
  tmp_out = {}
  res_out = {}
  for mark in marks:
    tmp_out[mark] = X.loc[:,[c_name for c_name in X.columns if c_name.startswith(mark)]]
    tmp_out[mark] = tmp_out[mark].div(tmp_out[mark].sum(axis=1), axis=0)
    res_out[mark + "_Polyfit"] = tmp_out[mark].apply(lambda y : find_slope_from_bins(y.to_numpy()), axis = 1)
    res_out[mark + "_maxBin"] = res_out[mark + "_Polyfit"].apply(lambda y : y[0])
    res_out[mark + "_upSlope"] = res_out[mark + "_Polyfit"].apply(lambda y : y[1])
    res_out[mark + "_downSlope"] = res_out[mark + "_Polyfit"].apply(lambda y : y[2])
    res_out[mark + "_fullSlope1"] = res_out[mark + "_Polyfit"].apply(lambda y : y[3])
    res_out[mark + "_fullSlope2"] = res_out[mark + "_Polyfit"].apply(lambda y : y[4])
  res_out = pd.DataFrame(res_out).drop(labels = [mark + "_Polyfit" for mark in marks], axis = 1)
  return pd.DataFrame(res_out)

get_polyfit(X.head(), marks) 

H3K36me3_maxBin  H3K36me3_upSlope  ...  H3K4me3_fullSlope1  H3K4me3_fullSlope2
0               94          0.002674  ...           -0.000008            0.000766
1               58          0.000859  ...           -0.000027            0.000464
2               33          0.001605  ...           -0.000025            0.000790
3                0          0.000000  ...            0.000109           -0.001866
4               63          0.002660  ...            0.000021           -0.000726

[5 rows x 25 columns]

#### Feature 5 : Correlation between marks

In [33]:
#!pip install pingouin

In [8]:
import pingouin as pg

def find_cor_from_bins(mybins, marks) :
  res_out = pd.DataFrame()
  df = pd.DataFrame()
  for mark in marks :
      df.reset_index()
      df[mark] = mybins[[mark + "_" + str(bin) for bin in range(0,10)]].to_numpy()
  try :
    dfcor = pg.pairwise_corr(df, method='pearson', padjust='none').loc[:, ['X','Y','r2','r']]
    column_name = dfcor["X"] + "::" + dfcor["Y"] + "pearson"
    dfcor = dfcor.loc[:,['r2','r']].T
    dfcor.columns = column_name
    res_out = pd.concat([res_out, dfcor], axis = 1)
    res_out_z = pd.DataFrame(res_out.iloc[1,:])
    res_out_z.index = [cn + "_R" for cn in res_out_z.index]
    res_out_z.columns = ['cor']
    res_out_r = pd.DataFrame(res_out.iloc[0,:])
    res_out_r.index = [cn + "_R2" for cn in res_out_r.index]
    res_out_r.columns = ['cor']
    res_out = pd.concat([res_out_r, res_out_z], axis = 0)
  except :
    res_out = pd.DataFrame()
    res_out['cor'] = [0] * 20
  res_out = pd.Series( res_out.loc[:,'cor'] )
  return res_out

def get_corpcor(X, marks):
  res_out = X.apply(lambda y : find_cor_from_bins(y, marks), axis = 1)
  return (res_out)

get_corpcor(X.head(), marks) 

H3K36me3::H3K27me3pearson_R2  ...  H3K9me3::H3K4me3pearson_R
0                      0.220078  ...                   0.485240
1                      0.457995  ...                   0.280056
2                      0.043737  ...                   0.027566
3                      0.272222  ...                   0.206897
4                      0.000445  ...                  -0.519291

[5 rows x 20 columns]

#### Create final train + test datasets

In [10]:
train_feat = pd.concat([get_global(X, marks), get_polyfit(X, marks), get_gini(X, marks), get_binom(X, marks), get_corpcor(X, marks)], axis = 1)
test_feat = pd.concat([get_global(test, marks), get_polyfit(test, marks), get_gini(test, marks), get_binom(test, marks), get_corpcor(test, marks)], axis = 1)

train_feat = train_feat.fillna(0)
train_feat.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.6/dist-packages/pingouin/effsize.py:135: RuntimeWarning: divide by zero encountered in arctanh
  z = np.arctanh(stat)  # R-to-z transform
/usr/local/lib/python3.6/dist-packages/pingouin/power.py:899: RuntimeWarning: divide by zero encountered in arctanh
  zr = np.arctanh(r) + r / (2 * (n - 1))
/usr/local/lib/python3.6/dist-packages/pingouin/pairwise.py:1274: RuntimeWarning: divide by zero encountered in arctanh
  stats['z'] = np.arctanh(stats['r'].to_numpy())
/usr/local/lib/python3.6/dist-packages/pingouin/effsize.py:135: RuntimeWarning: divide by zero encountered in arctanh
  z = np.arctanh(stat)  # R-to-z transform
/usr/local/lib/python3.6/dist-packages/pingouin/power.py:899: RuntimeWarning: divide by zero encountered in arctanh
  zr = np.arctanh(r) + r / (2 * (n - 1))
/usr/local/lib/python3.6/dist-packages/pingouin/pairwise.py:1274:

H3K36me3_sum  ...  H3K9me3::H3K4me3pearson_R2
0           720  ...                    0.235458
1            79  ...                    0.078431
2           356  ...                    0.000760
3            81  ...                    0.042806
4           243  ...                    0.269663

[5 rows x 135 columns]

In [11]:
test_feat = test_feat.fillna(0)
test_feat.head()

H3K36me3_sum  ...  H3K9me3::H3K4me3pearson_R2
0           661  ...                1.001224e-03
1           205  ...                7.670182e-03
2           212  ...                9.661836e-03
3           494  ...                4.814825e-35
4           476  ...                2.060696e-01

[5 rows x 135 columns]

In [12]:
# Verfify you have the same number of rows
print(train.shape)
print(train_feat.shape)
print(test.shape)
print(test_feat.shape)

(10436, 500)
(10436, 135)
(5049, 500)
(5049, 135)


##### Random Forest model

In [15]:
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from scipy.stats import randint
import xgboost as xgb

In [29]:
# Initialize a variable that will hold our model
rf_handle = RandomForestClassifier()

# Define parameter distributions
param_dist = {"n_estimators"      : [50],
              "max_depth"         : randint(1,50),
              "min_samples_split" : randint(3, 50),
              "min_samples_leaf"  : randint(3, 50),
              "bootstrap"         : [True, False]}

# Define the number of parameter sets to search for
n_iter_search = 10

# Initialize the randomized search cv
random_search = RandomizedSearchCV(rf_handle, param_distributions=param_dist,
                                   n_iter=n_iter_search,verbose=10,scoring="neg_log_loss", refit = True,
                                   n_jobs=8,cv=StratifiedKFold(n_splits=5, shuffle=True,random_state=42))

# Start the search for the best parameters, cv evaluation and final fitting on the best parameters
random_search = random_search.fit(train_feat, train_labels)

# Point to the best model with this variable
rf_model = random_search.best_estimator_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   13.4s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   30.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   35.5s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   52.7s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done  41 out of  50 | elapsed:  1.2min remaining:   16.2s
[Parallel(n_jobs=8)]: Done  47 out of  50 | elapsed:  1.4min remaining:    5.6s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:  1.5min finished


In [43]:
# Make predictions for the test set
predictions = rf_model.predict_proba(test_feat)

predictions_df = pd.DataFrame({"GeneId":test_index_col,"Label":predictions[:,1]})
predictions_df.to_csv('submission_rf.csv', index=False)

#### XGBoost model

In [21]:
from scipy.stats import uniform, randint

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric = "logloss")

params = {
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 10), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.8, 0.95)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params,
                            random_state=42, n_iter=100, cv=StratifiedKFold(n_splits=10, shuffle=True,random_state=42),
                            verbose=1, n_jobs=8, return_train_score=True,
                            scoring="neg_log_loss", refit = True)

search.fit(train_feat, train_labels)


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  49 tasks      | elapsed:  5.4min
[Parallel(n_jobs=8)]: Done 200 tasks      | elapsed: 11.7min
[Parallel(n_jobs=8)]: Done 450 tasks      | elapsed: 19.3min
[Parallel(n_jobs=8)]: Done 800 tasks      | elapsed: 31.9min
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed: 35.4min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
                   error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1,
                                           eval_metric='logloss', gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthre...
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa76ed71358>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa76ed7

In [22]:
# Point to the best model with this variable
xgb_model = search.best_estimator_

ax = xgb.plot_importance(xgb_model)
fig = ax.figure
fig.set_size_inches(10, 20)

In [24]:
# Make predictions for the test set
predictions = xgb_model.predict_proba(test_feat)

predictions_df = pd.DataFrame({"GeneId":test_index_col,"Label":predictions[:,1]})
predictions_df.to_csv('submission_xgb4.csv', index=False)